In [19]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from src.constants import PREPROCESSED_DATA_PATHS, SAMPLES_PATH

class SampleGenerator:
    def __init__(self, input_window_size, output_window_size, freq=30, output_averaging=False, input_averaging=False, use_log=False,
                 save=True):
        self.X = None
        self.Y = None

        self.input_window_size = input_window_size
        self.output_window_size = output_window_size
        self.freq = freq
        self.output_averaging = output_averaging
        self.input_averaging = input_averaging
        self.use_log = use_log
        self.save = save
        self.data_paths = "./data/4_preprocessed_data"

    def generate(self):
        sample_name = self._generate_sample_name()
#         if check_if_sample_exists(sample_name=sample_name):
#             print(f'Loading sample from backup: {sample_name}')
#             self._load_samples(sample_name=sample_name)
#         else:
        self._generate()
#         self._save_sample(sample_name=sample_name)
        return self.X, self.Y

    def _generate(self):
        X_df = pd.DataFrame()
        Y_df = pd.DataFrame()
        for path in self.data_paths:
            print(path)
            df = pd.read_csv(path, index_col=0)
            df.timestamp = pd.to_datetime(df.timestamp)
            df.sort_values(by='timestamp', inplace=True)
            if self.use_log:
                df.avg_price = np.log(df.avg_price)
            for i in range(0, df.shape[0], self.freq):
               
                if i > df.shape[0] - self.input_window_size - self.output_window_size:
                    break
                input_period = df.loc[i:i + self.input_window_size - 1, :].reset_index().avg_price
                output_period = df.loc[
                                i + self.input_window_size: i + self.input_window_size + self.output_window_size - 1, :] \
                    .reset_index().avg_price
                input_period_min = input_period.min()
                input_period_max = input_period.max()
                input_period = (input_period - input_period_min) / (input_period.max() - input_period_min)
                output_period = (output_period - input_period_min) / (input_period_max - input_period_min)
                print(output_period)
                X_df = X_df.append(input_period)
                Y_df = Y_df.append(output_period)

        return

        X_df = X_df.reset_index(drop=True)
        Y_df = Y_df.reset_index(drop=True)
        self.X = X_df.values
        self.Y = Y_df.values
        assert np.isnan(self.X).sum() == 0, 'Nans in X array'
        assert np.isnan(self.Y).sum() == 0, 'Nans in Y array'

    def _generate_sample_name(self):
        return f"""input_window_size_{self.input_window_size}_output_window_size_{self.output_window_size}_freq_{self.freq}_output_averaging_{self.output_averaging}_input_averaging_{self.input_averaging}_use_log_{self.use_log}_"""

sg = SampleGenerator(
    input_window_size=90,
    output_window_size=30,
    output_averaging=True
)
X, Y = sg.generate()

.


PermissionError: [Errno 13] Permission denied: '.'

In [49]:
output_period = pd.Series(range(0,90, 1))

In [50]:
freq=30
_output_period = pd.DataFrame(output_period)
_output_period['group'] = output_period // freq
output_period = _output_period.groupby('group').mean()[0]
output_period

group
0    14.5
1    44.5
2    74.5
Name: 0, dtype: float64

0    14.5
1    44.5
2    74.5
Name: 0, dtype: float64